In [ ]:
# Imports

import cv2
import numpy as np
import json
import os
import random
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout, Activation, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet import ResNet50
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import keras

In [ ]:
# Functions

def data_loader(path, data_name, img_size):
  all_images = []
  all_files = ['1004', '1005', '1015', '1017', '1019', '1021', '1039', '1040']
  for label_folder in all_files:
    inside_path = path+'/'+ label_folder +'/'+ data_name
    fold = os.listdir(inside_path)
    for img in fold:
      ext = os.path.splitext(img)[-1]
      if ext in ['.jpg', '.JPG']: # only jpg files no json files
        try:
          img_arr = cv2.imread(os.path.join(inside_path, img))
          new_arr = cv2.resize(img_arr, (img_size, img_size))
          all_images.append([new_arr, label_folder])
        except:
          pass 
    random.shuffle(all_images)
  return all_images


def preprocesing(data, num_class):
  X, y = [], []
  for features, label in data:
    X.append(features)                 
    y.append(label)  
  X = np.array(X).reshape(-1, 224, 224, 3)
  encoding = np.vectorize(label_dict.get)(y) # label encoding
  y = np.array(encoding)
  y = tf.keras.utils.to_categorical(y, num_classes=num_class) # one hot encoding
  print("The shape of the dataset:", X.shape)
  print("The shape of the label dataset:", y.shape)
  X = X/255.
  return X, y

In [ ]:
path = '/content/drive/MyDrive/CV_Images'

label_dict = {'1040':0, '1019':1, '1015':2, '1004':3, '1005':4, '1039':5, '1017':6,'1021':7,'1001':8,'1007':9,'1018':10}
classes  = len(label_dict)

# rename cropped_train to folder name which you want to run
# img_train = data_loader(path, 'train', 224)
# img_test_cropped = data_loader(path, 'cropped_test', 224)
# img_test_uncropped = data_loader(path, 'uncropped_test', 224)

# X_train, y_train = preprocesing(img_train, classes)
# X_test_cropped, y_test_cropped = preprocesing(img_test_cropped, classes)
# X_test_uncropped, y_test_uncropped = preprocesing(img_test_uncropped, classes)

X_train = np.load('/content/drive/MyDrive/CV_Images_npy_11/X_train.npy')
y_train = np.load('/content/drive/MyDrive/CV_Images_npy_11/y_train.npy')
X_test_cr = np.load('/content/drive/MyDrive/CV_Images_npy_11/X_test_cr.npy')
y_test_cr = np.load('/content/drive/MyDrive/CV_Images_npy_11/y_test_cr.npy')
X_test_uncr = np.load('/content/drive/MyDrive/CV_Images_npy_11/X_test_uncr.npy')
y_test_uncr = np.load('/content/drive/MyDrive/CV_Images_npy_11/y_test_uncr.npy')

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
momentum = 0.8
dp = 0.5

In [ ]:
def get_model():
    model = Sequential()

    model.add(Conv2D(128, (7, 7), input_shape = X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(Dropout(dp))

    model.add(Conv2D(128, (5, 5), input_shape = X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(Dropout(dp))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(Dropout(dp))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(Dropout(dp))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(Dropout(dp))

    model.add(Flatten())

    model.add(Dense(1024, activation='relu'))

    model.add(Dense(classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer="adam", 
                metrics=['accuracy'])

    return model

# Without Augmentation

In [ ]:
model = get_model()

In [ ]:
EPOCHS = 50

history = model.fit(X_train, y_train, batch_size=32, epochs=EPOCHS)

Epoch 1/50
28/28 [==============================] - 19s 178ms/step - loss: 1.7336 - accuracy: 0.4152
Epoch 2/50
28/28 [==============================] - 4s 140ms/step - loss: 0.7498 - accuracy: 0.7395
Epoch 3/50
28/28 [==============================] - 4s 140ms/step - loss: 0.5555 - accuracy: 0.8055
Epoch 4/50
28/28 [==============================] - 4s 143ms/step - loss: 0.3898 - accuracy: 0.8623
Epoch 5/50
28/28 [==============================] - 4s 144ms/step - loss: 0.3502 - accuracy: 0.8862
Epoch 6/50
28/28 [==============================] - 4s 144ms/step - loss: 0.2792 - accuracy: 0.8976
Epoch 7/50
28/28 [==============================] - 4s 144ms/step - loss: 0.2043 - accuracy: 0.9363
Epoch 8/50
28/28 [==============================] - 4s 145ms/step - loss: 0.1505 - accuracy: 0.9465
Epoch 9/50
28/28 [==============================] - 4s 143ms/step - loss: 0.1406 - accuracy: 0.9499
Epoch 10/50
28/28 [==============================] - 4s 144ms/step - loss: 0.1208 - accuracy: 0.959

In [ ]:
model.evaluate(X_test_cr, y_test_cr)

11/11 [==============================] - 3s 76ms/step - loss: 4.9254 - accuracy: 0.3865


[4.9253621101379395, 0.38650307059288025]

In [ ]:
model.evaluate(X_test_uncr, y_test_uncr)

11/11 [==============================] - 0s 36ms/step - loss: 5.6323 - accuracy: 0.2699


[5.632274627685547, 0.2699386477470398]

### Augmentation 1 (Default)

In [ ]:
datagen = ImageDataGenerator()
datagen.fit(X_train)

In [ ]:
model = get_model()

EPOCHS = 50

history = model.fit(datagen.flow(X_train, y_train), batch_size=32, epochs=EPOCHS)

Epoch 1/50
28/28 [==============================] - 5s 150ms/step - loss: 1.5779 - accuracy: 0.4881
Epoch 2/50
28/28 [==============================] - 4s 147ms/step - loss: 0.7929 - accuracy: 0.7304
Epoch 3/50
28/28 [==============================] - 4s 148ms/step - loss: 0.5321 - accuracy: 0.8077
Epoch 4/50
28/28 [==============================] - 4s 148ms/step - loss: 0.3751 - accuracy: 0.8510
Epoch 5/50
28/28 [==============================] - 4s 148ms/step - loss: 0.3693 - accuracy: 0.8703
Epoch 6/50
28/28 [==============================] - 4s 148ms/step - loss: 0.2508 - accuracy: 0.9124
Epoch 7/50
28/28 [==============================] - 4s 149ms/step - loss: 0.2148 - accuracy: 0.9204
Epoch 8/50
28/28 [==============================] - 4s 151ms/step - loss: 0.2944 - accuracy: 0.9067
Epoch 9/50
28/28 [==============================] - 4s 153ms/step - loss: 0.2404 - accuracy: 0.9170
Epoch 10/50
28/28 [==============================] - 4s 148ms/step - loss: 0.1247 - accuracy: 0.9556

Cropped Test

In [ ]:
model.evaluate(X_test_cr, y_test_cr)

11/11 [==============================] - 1s 36ms/step - loss: 6.0217 - accuracy: 0.3589


[6.0217485427856445, 0.3588957190513611]

Uncropped Test

In [ ]:
model.evaluate(X_test_uncr, y_test_uncr)

11/11 [==============================] - 0s 37ms/step - loss: 6.9866 - accuracy: 0.2638


[6.98657751083374, 0.2638036906719208]

### Augmentation 2

In [ ]:
datagen = ImageDataGenerator(horizontal_flip = True)
datagen.fit(X_train)

In [ ]:
model = get_model()

EPOCHS = 50

history = model.fit(datagen.flow(X_train, y_train), batch_size=32, epochs=EPOCHS)

Epoch 1/50
28/28 [==============================] - 5s 147ms/step - loss: 1.9483 - accuracy: 0.3800
Epoch 2/50
28/28 [==============================] - 4s 147ms/step - loss: 1.2850 - accuracy: 0.5802
Epoch 3/50
28/28 [==============================] - 4s 147ms/step - loss: 0.9831 - accuracy: 0.6621
Epoch 4/50
28/28 [==============================] - 4s 147ms/step - loss: 0.8250 - accuracy: 0.7315
Epoch 5/50
28/28 [==============================] - 4s 148ms/step - loss: 0.6534 - accuracy: 0.7861
Epoch 6/50
28/28 [==============================] - 4s 148ms/step - loss: 0.6244 - accuracy: 0.7929
Epoch 7/50
28/28 [==============================] - 4s 148ms/step - loss: 0.4462 - accuracy: 0.8544
Epoch 8/50
28/28 [==============================] - 4s 148ms/step - loss: 0.4303 - accuracy: 0.8692
Epoch 9/50
28/28 [==============================] - 4s 148ms/step - loss: 0.3447 - accuracy: 0.8874
Epoch 10/50
28/28 [==============================] - 4s 147ms/step - loss: 0.2851 - accuracy: 0.9010

Cropped Test

In [ ]:
model.evaluate(X_test_cr, y_test_cr)

11/11 [==============================] - 1s 44ms/step - loss: 2.9614 - accuracy: 0.6810


[2.961383819580078, 0.6809815764427185]

Uncropped Test

In [ ]:
model.evaluate(X_test_uncr, y_test_uncr)

11/11 [==============================] - 0s 37ms/step - loss: 5.3913 - accuracy: 0.4110


[5.391303539276123, 0.4110429584980011]

# Augmentation 3

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
datagen.fit(X_train)

In [ ]:
model = get_model()

EPOCHS = 50

history = model.fit(datagen.flow(X_train, y_train), batch_size=32, epochs=EPOCHS)

Epoch 1/50
28/28 [==============================] - 12s 380ms/step - loss: 2.1874 - accuracy: 0.2890
Epoch 2/50
28/28 [==============================] - 11s 379ms/step - loss: 1.8682 - accuracy: 0.4266
Epoch 3/50
28/28 [==============================] - 10s 358ms/step - loss: 1.5659 - accuracy: 0.5063
Epoch 4/50
28/28 [==============================] - 10s 358ms/step - loss: 1.4912 - accuracy: 0.5222
Epoch 5/50
28/28 [==============================] - 10s 359ms/step - loss: 1.3432 - accuracy: 0.5813
Epoch 6/50
28/28 [==============================] - 10s 357ms/step - loss: 1.1366 - accuracy: 0.6416
Epoch 7/50
28/28 [==============================] - 10s 360ms/step - loss: 0.9426 - accuracy: 0.6758
Epoch 8/50
28/28 [==============================] - 10s 358ms/step - loss: 0.9111 - accuracy: 0.7133
Epoch 9/50
28/28 [==============================] - 11s 375ms/step - loss: 0.7776 - accuracy: 0.7361
Epoch 10/50
28/28 [==============================] - 10s 358ms/step - loss: 0.7530 - accura

In [ ]:
model.evaluate(X_test_cr, y_test_cr)

11/11 [==============================] - 1s 36ms/step - loss: 736.1237 - accuracy: 0.0399


[736.1236572265625, 0.03987729921936989]

In [ ]:
model.evaluate(X_test_uncr, y_test_uncr)

11/11 [==============================] - 0s 36ms/step - loss: 754.8057 - accuracy: 0.0368


[754.8056640625, 0.03680981695652008]

# Augmentation 4

In [ ]:
datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

In [ ]:
model = get_model()

EPOCHS = 50

history = model.fit(datagen.flow(X_train, y_train), batch_size=32, epochs=EPOCHS)

Epoch 1/50
28/28 [==============================] - 11s 361ms/step - loss: 1.9993 - accuracy: 0.3618
Epoch 2/50
28/28 [==============================] - 10s 342ms/step - loss: 1.2343 - accuracy: 0.5836
Epoch 3/50
28/28 [==============================] - 10s 343ms/step - loss: 0.9603 - accuracy: 0.6815
Epoch 4/50
28/28 [==============================] - 10s 345ms/step - loss: 0.7656 - accuracy: 0.7543
Epoch 5/50
28/28 [==============================] - 10s 343ms/step - loss: 0.6509 - accuracy: 0.7895
Epoch 6/50
28/28 [==============================] - 10s 344ms/step - loss: 0.5137 - accuracy: 0.8339
Epoch 7/50
28/28 [==============================] - 10s 343ms/step - loss: 0.4447 - accuracy: 0.8498
Epoch 8/50
28/28 [==============================] - 10s 342ms/step - loss: 0.4613 - accuracy: 0.8441
Epoch 9/50
28/28 [==============================] - 10s 343ms/step - loss: 0.3905 - accuracy: 0.8623
Epoch 10/50
28/28 [==============================] - 10s 342ms/step - loss: 0.3088 - accura

In [ ]:
model.evaluate(X_test_cr, y_test_cr)

11/11 [==============================] - 1s 36ms/step - loss: 2.9942 - accuracy: 0.5031


[2.994241237640381, 0.5030674934387207]

In [ ]:
model.evaluate(X_test_uncr, y_test_uncr)

11/11 [==============================] - 0s 37ms/step - loss: 4.7160 - accuracy: 0.2883


[4.716043472290039, 0.28834354877471924]